Experiment with computer vision model

In [1]:
%load_ext autoreload
%autoreload 2
from dataclasses import MISSING, asdict, dataclass, field, fields
from typing import List

from multiobjective_opt.neural_net.models.cifar_parametrized import ResNet


@dataclass
class ResNetParams:
    num_blocks: List = field(
        default_factory=lambda: [2, 2, 2, 2]
    )  # Количество блоков в каждом слое
    num_filters: int = 32  # Количество фильтров в начальном слое
    use_batchnorm: bool = False  # Использовать BatchNorm
    use_dropout: bool = False  # Использовать Dropout
    dropout_prob: float = 0.5  # Вероятность дропаут

In [ ]:
a = ResNetParams()
model = ResNet(**asdict(a))
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)

2792810


In [3]:
models = []
model_params = []
a = ResNetParams(num_blocks=[8, 8, 8, 2], num_filters=64)
model = ResNet(**asdict(a))
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)
models.append(model)
model_params.append(asdict(a))

a = ResNetParams(
    num_blocks=[2, 2, 2, 4],
    num_filters=64,
)
model = ResNet(**asdict(a))
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)
models.append(model)
model_params.append(asdict(a))

a = ResNetParams(
    num_blocks=[1, 1, 1, 1],
    num_filters=128,
)
model = ResNet(**asdict(a))
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)
models.append(model)
model_params.append(asdict(a))

# with batch_norm
a = ResNetParams(num_blocks=[8, 8, 8, 2], num_filters=64, use_batchnorm=True)
model = ResNet(**asdict(a))
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)
models.append(model)
model_params.append(asdict(a))

a = ResNetParams(num_blocks=[2, 2, 2, 4], num_filters=64, use_batchnorm=True)
model = ResNet(**asdict(a))
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)
models.append(model)
model_params.append(asdict(a))

a = ResNetParams(num_blocks=[1, 1, 1, 1], num_filters=128, use_batchnorm=True)
model = ResNet(**asdict(a))
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)
models.append(model)
model_params.append(asdict(a))

# + dropout
a = ResNetParams(
    num_blocks=[8, 8, 8, 2], num_filters=64, use_batchnorm=True, use_dropout=True
)
model = ResNet(**asdict(a))
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)
models.append(model)
model_params.append(asdict(a))

a = ResNetParams(
    num_blocks=[2, 2, 2, 4], num_filters=64, use_batchnorm=True, use_dropout=True
)
model = ResNet(**asdict(a))
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)
models.append(model)
model_params.append(asdict(a))

a = ResNetParams(
    num_blocks=[1, 1, 1, 1], num_filters=128, use_batchnorm=True, use_dropout=True
)
model = ResNet(**asdict(a))
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)
models.append(model)
model_params.append(asdict(a))

20454090
20601546
19576202
20474442
20615242
19587722
20474442
20615242
19587722


In [4]:
from multiobjective_opt.neural_net.utils import funcs
from multiobjective_opt.neural_net.utils.dataset_prepare import CIFAR10Handler


dataloader_cycled = True
dataloader_iters = 100
num_pulls = 100
models_list = models
coeffs_list = []
for model in models:
    coeffs_list.append(None)


data_loader = CIFAR10Handler(dataloader_cycled, dataloader_iters, root="../data")
neural_ucb = funcs.UCB_nets(models_list, model_params, data_loader, coeffs_list)

# ucb_res = neural_ucb.ucb_train_models(sum_epochs=num_pulls)

Files already downloaded and verified
Files already downloaded and verified


/root/miniconda3/envs/MOO/lib/python3.11/site-packages/torch/autograd/graph.py:825: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


116.57396225747173 0.2546000559085041 29.67973730822815
29.67973730822815
Files already downloaded and verified
Files already downloaded and verified
95.67533640326288 0.15859698188074692 15.173819593982648
15.173819593982648
Files already downloaded and verified
Files already downloaded and verified
87.75566655933 0.17640578328857973 15.48060709741003
15.48060709741003
Files already downloaded and verified
Files already downloaded and verified
233.0059312909928 40.8454623724009 9517.234999092476
9517.234999092476
Files already downloaded and verified
Files already downloaded and verified
191.14924400301484 14.556653833583324 2782.49337550304
2782.49337550304
Files already downloaded and verified
Files already downloaded and verified
175.33079242784422 4.949168589466114 867.7416506500898
867.7416506500898
Files already downloaded and verified
Files already downloaded and verified
233.01037258012033 60.63104499695632 14127.662384662834
14127.662384662834
Files already downloaded and ver

In [5]:
from tabulate import tabulate


data = []

for model in ucb_res[0]:
    st = model.statistics
    accuracy = st.eval_results[-1]["accuracy"]
    loss = st.eval_results[-1]["loss"]
    confidence_interval = model.coeff / st.train_steps**0.5
    row = [st.model_name, st.num_pulls, accuracy, loss, confidence_interval]
    data.append(row)
headers = [
    "Model name",
    "model pulls",
    "Model Accuracy",
    "Model loss",
    "Confidence interval",
]
print(tabulate(data, headers=headers, floatfmt=".3f", tablefmt="heavy_outline"))

NameError: name 'ucb_res' is not defined

In [6]:
for i in range(len(neural_ucb.model_arms)):
    print(neural_ucb.model_arms[i].coeff)

21.20087001932637
14.458671496000386
15.413680719532742
9623.514999396855
3004.5635911669156
917.5321898103019
13927.537592967286
3594.5784040607928
1145.6596388311993
